In [2]:
import pathlib
import delta
import json


Let's check if we will work on CPU or GPU:

In [3]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

Set the path to the one you used in `download_test_data_delta`

In [3]:
root = pathlib.Path(pathlib.Path.home(), 
                    'I2ICourse/RawData/project1')

output_dir = pathlib.Path(pathlib.Path.home(), 
                    'I2ICourse/ProcessedData/project1')

(output_dir).mkdir(exist_ok=True)


We need to modify the config file for the proper paths

In [4]:
# Load config ('2D' or 'mothermachine'):
config_file = str((root / 'config_2D.json').resolve())

with open(config_file) as f:
    data = json.load(f)
    
old_model_path = 'D:/DeLTA_data/agar_pads'  
new_model_path = str((root / 'models').resolve())
data['model_file_seg'] = data['model_file_seg'].replace(old_model_path, new_model_path)    
data['model_file_track'] = data['model_file_track'].replace(old_model_path, new_model_path)

data['training_set_seg'] = str((root / 'sets' / '2D' / 'training' / 'segmentation_set').resolve())
data['training_set_track'] = str((root / 'sets' / '2D' / 'training' / 'tracking_set').resolve())

data['eval_movie'] = str((root / 'evals' / '2D' / 'movie_tifs').resolve())

data['save_format'] = ["pickle", "movie"]   

new_config_file = str((root / 'config_2D_local.json').resolve())
with open(new_config_file, 'w') as f:
    json.dump(data, f, indent=2)
    

Now we setup the pipeline

In [9]:
# Load config ('2D' or 'mothermachine'):
delta.config.load_config(new_config_file)

# Use eval movie as example (or replace by None for interactive selection):
file_path = delta.config.eval_movie

# Init reader (use bioformats=True if working with nd2, czi, ome-tiff etc):
xpreader = delta.utils.xpreader(file_path)

# Print experiment parameters to make sure it initialized properly:
print("""Initialized experiment reader:
    - %d positions
    - %d imaging channels
    - %d timepoints"""%(xpreader.positions, xpreader.channels, xpreader.timepoints)
)

# Init pipeline:
xp = delta.pipeline.Pipeline(xpreader, resfolder=output_dir)

Loading configuration from: /Users/simonvanvliet/I2ICourse/RawData/project1/config_2D_local.json
Initialized experiment reader:
    - 1 positions
    - 1 imaging channels
    - 74 timepoints


Running the full analysis without a GPU takes a long time (150min on 2017 MacBook Pro with 2.8 GHz Quad-Core i7; using a RTX8000 GPU the same dataset is however processed in 3min!).

Here we will just analyze the first 4 frames, so you can see the process. We will then load pre-processed data in the next notebook.

In [11]:
# Run it (you can specify which positions, which frames to run etc):
xp.process(frames=list(range(4)))
#

Wed Mar  9 09:03:19 2022, Position 0 - Starting pre-processing
Wed Mar  9 09:03:20 2022, Position 0 - Starting segmentation (4 frames)
Wed Mar  9 09:08:20 2022, Position 0 - Starting tracking (4 frames)
Wed Mar  9 09:08:20 2022, Position 0 - Tracking - frame 0/4 
Wed Mar  9 09:08:20 2022, Position 0 - Tracking - frame 1/4 
Wed Mar  9 09:08:22 2022, Position 0 - Tracking - frame 2/4 
Wed Mar  9 09:08:24 2022, Position 0 - Tracking - frame 3/4 
Wed Mar  9 09:08:25 2022, Position 0 - Starting feature extraction (4 frames)
Wed Mar  9 09:08:25 2022, Position 0 - Feature extraction - frame 0/4 
Wed Mar  9 09:08:26 2022, Position 0 - Feature extraction - frame 1/4 
Wed Mar  9 09:08:26 2022, Position 0 - Feature extraction - frame 2/4 
Wed Mar  9 09:08:26 2022, Position 0 - Feature extraction - frame 3/4 
Wed Mar  9 09:08:26 2022, Position 0 - Saving to legacy format
/Users/simonvanvliet/I2ICourse/ProcessedData/project1/Position000000.mat
Wed Mar  9 09:08:27 2022, Position 0 - Saving to pickle